# 🤖 Tutorial 4: ReAct Agent - Autonomous Compliance Verification

## 🎯 What You'll Learn

In this tutorial, you'll see how all the components from the previous tutorials work together in a **ReAct Agent** that can:

1. **🧠 Reason** about building compliance problems autonomously
2. **🔧 Use tools** to access data, perform calculations, and query regulations
3. **📚 Query documents** using the RAG system from Tutorial 3
4. **🏗️ Analyze buildings** using calculations from Tutorial 2
5. **💬 Have conversations** about compliance in natural language

## 📋 Table of Contents

1. [What is a ReAct Agent?](#1-what-is-a-react-agent)
2. [Setting Up the Agent](#2-setting-up-the-agent)
3. [Understanding the Tools](#3-understanding-the-tools)
4. [Testing Simple Queries](#4-testing-simple-queries)
5. [Complex Compliance Verification](#5-complex-compliance-verification)
6. [Conversational Interface](#6-conversational-interface)

---


## 1. What is a ReAct Agent?

**ReAct** = **Rea**soning + **Act**ing

A ReAct agent is an AI system that can:
1. 🧠 **Think** about what it needs to do
2. 🔧 **Choose** which tools to use
3. ⚡ **Execute** the tools
4. 📊 **Observe** the results
5. 🔁 **Repeat** until it has the answer

### How it works:

```
User: "Check if all doors meet requirements"
     ↓
Agent: "I need to list all doors first"
     ↓
Tool: list_all_doors()
     ↓
Agent: "Now I'll check each door's compliance"
     ↓
Tool: check_door_width_compliance("D001")
     ↓
Agent: "Let me query the regulations for requirements"
     ↓
Tool: query_normativa("minimum door width")
     ↓
Agent: "Here's my final answer..."
```

The agent **autonomously** decides what to do and uses the right tools to get the information it needs!


In [ ]:
# Create a streaming version that shows the agent's reasoning
def chat_with_agent_streaming(question: str) -> str:
    """
    Chat with the ReAct agent showing real-time reasoning and tool usage.
    
    Args:
        question: The question to ask the agent
    
    Returns:
        String with the agent's response
    """
    # Load project data if not already loaded
    from src.agent.tools import load_project_data
    project_file = Path("data/extracted/tutorial_example.json")
    if project_file.exists():
        load_project_data(project_file)
    
    # Use the simple agent's streaming method
    try:
        result = agent.run_with_streaming(question, max_iterations=10)
        return result['final_answer']
    except Exception as e:
        print(f"\n❌ Error running agent: {str(e)}")
        return f"Error: {str(e)}"

print("✅ Streaming interface created")
print("🔍 Use chat_with_agent_streaming('your question') for live reasoning")


## 2. Setting Up the Agent

Let's set up our ReAct agent using the components we've already built!


In [1]:
# Import required libraries
import sys
import os
from pathlib import Path

# Ensure project root is on sys.path so `src` is importable, even if kernel starts in notebooks/
ROOT = Path.cwd()
try:
    # Walk up until we find a folder containing `src`
    while ROOT != ROOT.parent and not (ROOT / 'src').exists():
        ROOT = ROOT.parent
finally:
    if (ROOT / 'src').exists() and str(ROOT) not in sys.path:
        sys.path.insert(0, str(ROOT))

# Test LangChain installation
print("🔍 Testing LangChain installation...")
try:
    import langchain_core
    print(f"✅ LangChain Core version: {langchain_core.__version__}")
except ImportError as e:
    print(f"❌ LangChain Core not found: {e}")
    print("Try: pip install langchain-core")

# Load environment variables from .env file
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ Environment variables loaded from .env file")
except ImportError:
    print("⚠️ python-dotenv not installed, trying to load .env manually")
    try:
        import os
        env_file = Path(".env")
        if env_file.exists():
            with open(env_file) as f:
                for line in f:
                    if line.strip() and not line.startswith("#"):
                        key, value = line.strip().split("=", 1)
                        os.environ[key] = value
            print("✅ Environment variables loaded from .env file")
        else:
            print("⚠️ No .env file found")
    except Exception as e:
        print(f"⚠️ Could not load .env file: {e}")

# Import LangChain components
try:
    from langchain_core.messages import HumanMessage
    print("✅ HumanMessage imported successfully")
    
    # Test creating a message
    test_msg = HumanMessage(content="Test message")
    print("✅ HumanMessage creation test passed")
except ImportError as e:
    print(f"❌ Error importing HumanMessage: {e}")
    print("Try: pip install langchain-core")
except Exception as e:
    print(f"❌ Error with HumanMessage: {e}")

# Import our existing agent components
try:
    from src.agent.simple_agent import create_simple_compliance_agent
    from src.agent.tools import load_project_data, set_vectorstore_manager
    print("✅ Agent components imported successfully")
    print("🔧 Using Simple ReAct Agent (LangGraph-free implementation)")
except ImportError as e:
    print(f"❌ Error importing agent components: {e}")
    
    # Provide specific installation instructions for missing packages
    if "No module named 'langchain_openai'" in str(e):
        print("\n🔧 Missing Dependency: langchain-openai")
        print("   Install with: pip install langchain-openai")
    elif "No module named 'langchain_community'" in str(e):
        print("\n🔧 Missing Dependency: langchain-community")
        print("   Install with: pip install langchain-community")
    elif "No module named 'langchain_huggingface'" in str(e):
        print("\n🔧 Missing Dependency: langchain-huggingface")
        print("   Install with: pip install langchain-huggingface")
    else:
        print("\n🔧 Try installing all dependencies:")
        print("   pip install -r requirements.txt")

print("🤖 Ready to use our ReAct agent")


🔍 Testing LangChain installation...
✅ LangChain Core version: 1.0.0
✅ Environment variables loaded from .env file
✅ HumanMessage imported successfully
✅ HumanMessage creation test passed
✅ Agent components imported successfully
🔧 Using Simple ReAct Agent (LangGraph-free implementation)
🤖 Ready to use our ReAct agent


In [2]:
# Load project data (use example dataset from Tutorial 1/2)
print("\n📁 Loading project data...")

# Try different possible paths for the project file
possible_paths = [
    Path("data/extracted/tutorial_example.json"),
    Path("../data/extracted/tutorial_example.json"),
    Path("../../data/extracted/tutorial_example.json"),
    ROOT / "data/extracted/tutorial_example.json"
]

project_file = None
for path in possible_paths:
    if path.exists():
        project_file = path
        break

if project_file is None:
    print(f"❌ Project file not found in any expected location")
    print("Available files in data/extracted/:")
    data_dir = Path("data/extracted")
    if data_dir.exists():
        for f in data_dir.glob("*.json"):
            print(f"   - {f.name}")
    else:
        print("   No data/extracted directory found")
    print("\n💡 You can use any of the available JSON files by updating the path above")
else:
    try:
        load_project_data(project_file)
        print(f"✅ Project data loaded: {project_file}")
    except Exception as e:
        print(f"❌ Error loading project data: {e}")
        print("Make sure the file exists and is valid JSON")

# Setup RAG for query_normativa tool if available
print("\n📚 Setting up RAG system...")
rag_available = False
vectorstore_path = Path("vectorstore/normativa_db")

if vectorstore_path.exists() and any(vectorstore_path.iterdir()):
    try:
        from src.rag.vectorstore_manager import VectorstoreManager
        from src.rag.qa_chain import create_qa_chain
        
        rag = VectorstoreManager(vectorstore_path)
        rag.load_existing()
        retriever = rag.get_retriever(k=3)
        qa_chain = create_qa_chain(retriever)

        class RAGWrapper:
            def query(self, question):
                return qa_chain({"query": question})

        set_vectorstore_manager(RAGWrapper())
        rag_available = True
        print("✅ RAG system connected")
    except Exception as e:
        print("⚠️ Could not initialize RAG:", e)
        print("   Tip: Create the vectorstore before the demo.")
        print("   For setup, see docs or run: python scripts/create_vectorstore.py")
else:
    print("ℹ️ RAG vectorstore not found.")
    
    # Try to create the vectorstore automatically
    normativa_dir = Path("data/normativa")
    if normativa_dir.exists() and any(normativa_dir.glob("*.pdf")):
        print("📄 Found PDF documents in data/normativa/")
        print("🔄 Attempting to create vectorstore automatically...")
        try:
            import subprocess
            result = subprocess.run(["python", "scripts/create_vectorstore.py"], 
                                  capture_output=True, text=True, timeout=60)
            if result.returncode == 0:
                print("✅ Vectorstore created successfully!")
                # Try to load it now
                try:
                    from src.rag.vectorstore_manager import VectorstoreManager
                    from src.rag.qa_chain import create_qa_chain
                    
                    rag = VectorstoreManager(vectorstore_path)
                    rag.load_existing()
                    retriever = rag.get_retriever(k=3)
                    qa_chain = create_qa_chain(retriever)

                    class RAGWrapper:
                        def query(self, question):
                            return qa_chain({"query": question})

                    set_vectorstore_manager(RAGWrapper())
                    rag_available = True
                    print("✅ RAG system connected")
                except Exception as e:
                    print(f"⚠️ Could not load newly created vectorstore: {e}")
            else:
                print(f"❌ Failed to create vectorstore: {result.stderr}")
                print("   You can try manually: python scripts/create_vectorstore.py")
        except subprocess.TimeoutExpired:
            print("⏰ Vectorstore creation timed out. You can try manually: python scripts/create_vectorstore.py")
        except Exception as e:
            print(f"⚠️ Could not create vectorstore automatically: {e}")
            print("   You can try manually: python scripts/create_vectorstore.py")
    else:
        print("   No PDF documents found in data/normativa/")
        print("   To enable RAG, add PDF files to data/normativa/ and run: python scripts/create_vectorstore.py")

# Create agent
print("\n🤖 Creating ReAct agent...")

# Check if API key is available
import os
if os.getenv("OPENAI_API_KEY"):
    print("✅ OpenAI API key found")
    try:
        agent = create_simple_compliance_agent()
        print("✅ Simple ReAct Agent created successfully")
        print("🔧 This agent handles tool calls correctly without LangGraph issues")
        print("\n🎉 Ready to verify compliance!")
    except Exception as e:
        print(f"❌ Error creating agent: {e}")
        print("\n🔧 Troubleshooting:")
        print("  - Version compatibility: Try: pip install --upgrade langchain-openai langchain-core")
        print("  - Missing dependencies: pip install -r requirements.txt")
        
        # Check for specific issues
        if "cannot import name 'content'" in str(e):
            print("\n⚠️ Version Compatibility Issue Detected:")
            print("  This is a known issue with langchain-openai 1.0.0 and langchain-core 0.3.79")
            print("  Try: pip install langchain-openai==0.2.8")
            print("  Or: pip install --upgrade langchain-openai langchain-core")
        elif "No module named" in str(e):
            print("\n⚠️ Missing Dependencies Detected")
            print("  Try: pip install -r requirements.txt")
            print("  Or install individual packages as shown above")
        
        print("\n💡 For now, you can still test the individual tools below!")
else:
    print("⚠️ OpenAI API key not found")
    print("   Make sure your .env file contains: OPENAI_API_KEY=your-key-here")
    print("   Or set it as environment variable: export OPENAI_API_KEY=your-key-here")
    print("\n💡 For now, you can still test the individual tools below!")



📁 Loading project data...
✅ Project data loaded: ../data/extracted/tutorial_example.json

📚 Setting up RAG system...
ℹ️ RAG vectorstore not found.
   No PDF documents found in data/normativa/
   To enable RAG, add PDF files to data/normativa/ and run: python scripts/create_vectorstore.py

🤖 Creating ReAct agent...
✅ OpenAI API key found
✅ Simple ReAct Agent created successfully
🔧 This agent handles tool calls correctly without LangGraph issues

🎉 Ready to verify compliance!


## 3. Understanding the Tools

Our ReAct agent has access to 6 powerful tools that combine all the functionality from the previous tutorials:

### 🏗️ **Data Tools** (from Tutorial 1)
- **`get_room_info(room_id)`** - Get detailed room information
- **`get_door_info(door_id)`** - Get detailed door information  
- **`list_all_doors()`** - List all doors in the project

### 📐 **Calculation Tools** (from Tutorial 2)
- **`check_door_width_compliance(door_id)`** - Check door width compliance
- **`calculate_egress_distance(room_id)`** - Calculate evacuation distances

### 📚 **RAG Tool** (from Tutorial 3)
- **`query_normativa(question)`** - Query Spanish building codes (CTE)

Let's test these tools individually first to make sure they work!


In [3]:
# Show available tools
print("🛠️ Available Agent Tools")
print("=" * 50)

from src.agent.tools import get_available_tools

tools_info = get_available_tools()
for i, tool in enumerate(tools_info, 1):
    print(f"\n{i}. {tool['name']}")
    print(f"   Description: {tool['description']}")
    print(f"   Parameters: {', '.join(tool['parameters'])}")

print(f"\n📊 Total tools available: {len(tools_info)}")

# Test the tools individually first
print("\n🧪 Testing Tools Individually")
print("=" * 50)

from src.agent.tools import (
    get_room_info, get_door_info, list_all_doors,
    check_door_width_compliance, query_normativa, calculate_egress_distance
)

# Test 1: List all doors
print("\n1. Testing list_all_doors.invoke({}):")
try:
    doors = list_all_doors.invoke({})
    if isinstance(doors, list) and len(doors) > 0:
        print(f"   Found {len(doors)} doors:")
        for door in doors[:3]:  # Show first 3
            print(f"   - {door.get('id', 'Unknown')}: {door.get('name', 'N/A')}")
        if len(doors) > 3:
            print(f"   ... and {len(doors) - 3} more")
        print("   ✅ list_all_doors.invoke({}) works!")
    else:
        print("   ⚠️ No doors found or empty response")
except Exception as e:
    print(f"   ❌ Error: {e}")

# Test 2: Get room info
print("\n2. Testing get_room_info():")
try:
    room_info = get_room_info.invoke({"room_id": "R001"})
    if "error" not in room_info:
        print(f"   Room: {room_info.get('name', 'Unknown')}")
        print(f"   Area: {room_info.get('area_sqm', 'Unknown')} m²")
        print(f"   Use: {room_info.get('use', 'Unknown')}")
        print("   ✅ get_room_info() works!")
    else:
        print(f"   ❌ Error: {room_info['error']}")
except Exception as e:
    print(f"   ❌ Error: {e}")

# Test 3: Check door compliance
print("\n3. Testing check_door_width_compliance():")
try:
    compliance = check_door_width_compliance.invoke({"door_id": "D001"})
    if "error" not in compliance:
        print(f"   Door: {compliance.get('door_id', 'Unknown')}")
        print(f"   Width: {compliance.get('clear_width_mm', 'Unknown')} mm")
        print(f"   Status: {compliance.get('compliance_status', 'Unknown')}")
        print("   ✅ check_door_width_compliance() works!")
    else:
        print(f"   ❌ Error: {compliance['error']}")
except Exception as e:
    print(f"   ❌ Error: {e}")

# Test 4: RAG functionality (if available)
print("\n4. Testing query_normativa() (RAG):")
if rag_available:
    try:
        rag_result = query_normativa.invoke({"question": "What are the minimum door width requirements?"})
        if "error" not in rag_result:
            print(f"   Question: {rag_result.get('question', 'N/A')}")
            print(f"   Answer: {rag_result.get('answer', 'N/A')[:100]}...")
            print(f"   Sources: {len(rag_result.get('sources', []))} found")
            print("   ✅ query_normativa() works!")
        else:
            print(f"   ❌ Error: {rag_result['error']}")
    except Exception as e:
        print(f"   ❌ Error: {e}")
        if "cannot import name 'content'" in str(e):
            print("   💡 This is the same version compatibility issue as above")
else:
    print("   ⚠️ RAG not available - skipping test")

print("\n✅ Tool testing complete!")

# Check if agent is available for the next sections
if 'agent' not in locals():
    print("\n⚠️ Note: Agent not available.")
    if 'project_file' in locals() and project_file:
        print("   ✅ Project data is loaded, so the individual tools will work perfectly!")
        print("   💡 Set your OpenAI API key to test the full ReAct agent.")
    else:
        print("   The individual tools work fine, but the ReAct agent needs setup.")
        print("   You can still see how the tools work individually above!")


🛠️ Available Agent Tools

1. get_room_info
   Description: Get detailed information about a room including area, use, and geometric properties
   Parameters: room_id: str

2. get_door_info
   Description: Get detailed information about a door including dimensions and compliance data
   Parameters: door_id: str

3. list_all_doors
   Description: List all doors in the project with basic information
   Parameters: 

4. check_door_width_compliance
   Description: Check if a door meets minimum width requirements for compliance
   Parameters: door_id: str

5. query_normativa
   Description: Query building codes and regulations using RAG system
   Parameters: question: str

6. calculate_egress_distance
   Description: Calculate evacuation distance from a room to the nearest exit
   Parameters: room_id: str

📊 Total tools available: 6

🧪 Testing Tools Individually

1. Testing list_all_doors.invoke({}):
   Found 4 doors:
   - D001: Puerta Principal
   - D002: Puerta Recepción-Oficina
   - D003:

## 4. Agent Memory and State

Our ReAct agent includes **memory capabilities** that help it:

- 🧠 **Remember previous interactions** in the conversation
- 📝 **Track compliance results** across multiple checks
- 🔄 **Maintain context** between tool calls
- 📊 **Summarize findings** when reaching iteration limits

The agent uses a **SlidingWindowMemory** that keeps the most recent interactions while summarizing older ones to stay within token limits.

### 🔧 **Version Compatibility Fix**

If you encountered the "cannot import name 'content'" error, this is a known compatibility issue between `langchain-openai` 1.0.0 and `langchain-core` 0.3.79. Here are the solutions:

**Option 1: Downgrade langchain-openai**
```bash
pip install langchain-openai==0.2.8
```

**Option 2: Upgrade both packages**
```bash
pip install --upgrade langchain-openai langchain-core
```

**Option 3: Use the tools directly** (as shown in the previous section)

Now let's test the agent with simple queries to see it in action!


In [4]:
# Fix: Test tools correctly using .invoke() method
print("🔧 CORRECTED: Testing Tools with .invoke() Method")
print("=" * 60)

# Test 1: List all doors
print("\n1. Testing list_all_doors.invoke({}):")
try:
    doors = list_all_doors.invoke({})
    if isinstance(doors, list) and len(doors) > 0:
        print(f"   Found {len(doors)} doors:")
        for door in doors[:3]:  # Show first 3
            print(f"   - {door.get('id', 'Unknown')}: {door.get('name', 'N/A')}")
        if len(doors) > 3:
            print(f"   ... and {len(doors) - 3} more")
        print("   ✅ list_all_doors.invoke({}) works!")
    else:
        print("   ⚠️ No doors found or empty response")
except Exception as e:
    print(f"   ❌ Error: {e}")

# Test 2: Get room info
print("\n2. Testing get_room_info():")
try:
    room_info = get_room_info.invoke({"room_id": "R001"})
    if "error" not in room_info:
        print(f"   Room: {room_info.get('name', 'Unknown')}")
        print(f"   Area: {room_info.get('area_sqm', 'Unknown')} m²")
        print(f"   Use: {room_info.get('use', 'Unknown')}")
        print("   ✅ get_room_info() works!")
    else:
        print(f"   ❌ Error: {room_info['error']}")
except Exception as e:
    print(f"   ❌ Error: {e}")

# Test 3: Check door compliance
print("\n3. Testing check_door_width_compliance():")
try:
    compliance = check_door_width_compliance.invoke({"door_id": "D001"})
    if "error" not in compliance:
        print(f"   Door: {compliance.get('door_id', 'Unknown')}")
        print(f"   Compliant: {compliance.get('is_compliant', 'Unknown')}")
        print(f"   Width: {compliance.get('actual_width_mm', 'Unknown')} mm")
        print("   ✅ check_door_width_compliance() works!")
    else:
        print(f"   ❌ Error: {compliance['error']}")
except Exception as e:
    print(f"   ❌ Error: {e}")

# Test 4: Query normativa (RAG)
print("\n4. Testing query_normativa() (RAG):")
try:
    if 'vectorstore_manager' in locals() and vectorstore_manager:
        normativa_result = query_normativa.invoke({"question": "What are the minimum door width requirements?"})
        if "error" not in normativa_result:
            print(f"   Query: {normativa_result.get('query', 'Unknown')}")
            print(f"   Answer: {normativa_result.get('answer', 'Unknown')[:100]}...")
            print("   ✅ query_normativa() works!")
        else:
            print(f"   ❌ Error: {normativa_result['error']}")
    else:
        print("   ⚠️ RAG not available - skipping test")
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n✅ Tool testing complete!")
print("\n💡 Note: LangChain tools must be called with .invoke() method, not as functions!")


🔧 CORRECTED: Testing Tools with .invoke() Method

1. Testing list_all_doors.invoke({}):
   Found 4 doors:
   - D001: Puerta Principal
   - D002: Puerta Recepción-Oficina
   - D003: Puerta Sala Reuniones
   ... and 1 more
   ✅ list_all_doors.invoke({}) works!

2. Testing get_room_info():
   Room: Recepción
   Area: 20.0 m²
   Use: reception
   ✅ get_room_info() works!

3. Testing check_door_width_compliance():
   Door: D001
   Compliant: True
   Width: Unknown mm
   ✅ check_door_width_compliance() works!

4. Testing query_normativa() (RAG):
   ⚠️ RAG not available - skipping test

✅ Tool testing complete!

💡 Note: LangChain tools must be called with .invoke() method, not as functions!


## 🔧 **Important: LangChain Tool Usage**

### **❌ Common Mistake**
```python
# This will NOT work - tools are not regular functions
doors = list_all_doors()  # Error: 'StructuredTool' object is not callable
```

### **✅ Correct Usage**
```python
# LangChain tools must be called with .invoke() method
doors = list_all_doors.invoke({})  # Empty dict for no parameters
room_info = get_room_info.invoke({"room_id": "R001"})  # Dict with parameters
```

### **📝 Tool Parameter Format**
- **No parameters**: `tool.invoke({})`
- **With parameters**: `tool.invoke({"param_name": "value"})`
- **Multiple parameters**: `tool.invoke({"param1": "value1", "param2": "value2"})`

### **🎯 Why This Matters**
- **LangChain tools** are `StructuredTool` objects, not regular Python functions
- **The agent** automatically handles the `.invoke()` calls when using tools
- **Manual testing** requires using `.invoke()` method explicitly


In [5]:
# Example 1: List all doors
print("="*70)
print("EXAMPLE 1: List all doors")
print("="*70 + "\n")

print("🤖 Agent reasoning process:")
print("1. User asks: 'List all doors in the project'")
print("2. Agent thinks: 'I need to use the list_all_doors tool'")
print("3. Agent calls: list_all_doors.invoke({})")
print("4. Agent receives: List of doors")
print("5. Agent responds: Formatted door list")
print("\n" + "="*50)

if 'agent' in locals():
    result = agent.invoke({
        "messages": [HumanMessage(content="List all doors in the project")],
        "iterations": 0
    })

    # Print agent's response
    print("\n🤖 Agent's actual response:")
    print(f"📝 {result['final_answer']}")
    print(f"\n✅ Completed in {result['iterations']} iterations")
    print(f"🔧 Simple agent handles tool calls correctly!")
else:
    print("\n⚠️ Agent not available due to setup issues.")
    print("   But we can still demonstrate the tool directly:")
    print("\n🔧 Direct tool call:")
    doors = list_all_doors.invoke({})
    print(f"Found {len(doors)} doors:")
    for door in doors[:3]:
        print(f"  - {door.get('id', 'Unknown')}: {door.get('name', 'N/A')}")
    if len(doors) > 3:
        print(f"  ... and {len(doors) - 3} more")


EXAMPLE 1: List all doors

🤖 Agent reasoning process:
1. User asks: 'List all doors in the project'
2. Agent thinks: 'I need to use the list_all_doors tool'
3. Agent calls: list_all_doors.invoke({})
4. Agent receives: List of doors
5. Agent responds: Formatted door list


🤖 Agent's actual response:
📝 Here is the list of all doors in the project:

1. Door ID: D001
   - Name: Puerta Principal
   - Width: 900 mm
   - Type: Single door
   - From Room: EXTERIOR
   - To Room: R001
   - Emergency Exit: Yes
   - Fire Rating: RF_60

2. Door ID: D002
   - Name: Puerta Recepción-Oficina
   - Width: 800 mm
   - Type: Single door
   - From Room: R001
   - To Room: R002
   - Emergency Exit: No
   - Fire Rating: RF_60

3. Door ID: D003
   - Name: Puerta Sala Reuniones
   - Width: 800 mm
   - Type: Single door
   - From Room: R001
   - To Room: R003
   - Emergency Exit: No
   - Fire Rating: RF_60

4. Door ID: D004
   - Name: Puerta Almacén
   - Width: 800 mm
   - Type: Single door
   - From Room: R002

In [6]:
# Example 2: Check door compliance
print("="*70)
print("EXAMPLE 2: Check door compliance")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="Check if all doors meet minimum width requirements")],
    "iterations": 0
})

# Print agent's reasoning and answer
for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)
        print()

print(f"✅ Completed in {result['iterations']} iterations")


EXAMPLE 2: Check door compliance

Check if all doors meet minimum width requirements

[{'id': 'D001', 'name': 'Puerta Principal', 'width_mm': 900.0, 'door_type': 'single', 'from_room': 'EXTERIOR', 'to_room': 'R001', 'is_emergency_exit': True, 'fire_rating': 'RF_60'}, {'id': 'D002', 'name': 'Puerta Recepción-Oficina', 'width_mm': 800.0, 'door_type': 'single', 'from_room': 'R001', 'to_room': 'R002', 'is_emergency_exit': False, 'fire_rating': 'RF_60'}, {'id': 'D003', 'name': 'Puerta Sala Reuniones', 'width_mm': 800.0, 'door_type': 'single', 'from_room': 'R001', 'to_room': 'R003', 'is_emergency_exit': False, 'fire_rating': 'RF_60'}, {'id': 'D004', 'name': 'Puerta Almacén', 'width_mm': 800.0, 'door_type': 'single', 'from_room': 'R002', 'to_room': 'R004', 'is_emergency_exit': False, 'fire_rating': 'RF_30'}]

{'door_id': 'D001', 'door_type': 'single', 'clear_width_mm': 900.0, 'required_width_mm': 900, 'is_compliant': True, 'compliance_status': 'COMPLIANT', 'message': 'Door D001 meets minimum 

## 5. Complex Compliance Verification

Now let's see the agent handle more complex tasks that require multiple tools and reasoning!


In [7]:
# Example 3: Complex compliance check
print("="*70)
print("EXAMPLE 3: Complex Compliance Check")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="""
    Perform a comprehensive compliance check:
    1. List all doors
    2. Check each door's width compliance
    3. Calculate evacuation distances from all rooms
    4. Summarize findings with specific non-compliant items
    """)],
    "iterations": 0
})

for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)
        print()

print(f"✅ Completed in {result['iterations']} iterations")


EXAMPLE 3: Complex Compliance Check


    Perform a comprehensive compliance check:
    1. List all doors
    2. Check each door's width compliance
    3. Calculate evacuation distances from all rooms
    4. Summarize findings with specific non-compliant items
    

[{'id': 'D001', 'name': 'Puerta Principal', 'width_mm': 900.0, 'door_type': 'single', 'from_room': 'EXTERIOR', 'to_room': 'R001', 'is_emergency_exit': True, 'fire_rating': 'RF_60'}, {'id': 'D002', 'name': 'Puerta Recepción-Oficina', 'width_mm': 800.0, 'door_type': 'single', 'from_room': 'R001', 'to_room': 'R002', 'is_emergency_exit': False, 'fire_rating': 'RF_60'}, {'id': 'D003', 'name': 'Puerta Sala Reuniones', 'width_mm': 800.0, 'door_type': 'single', 'from_room': 'R001', 'to_room': 'R003', 'is_emergency_exit': False, 'fire_rating': 'RF_60'}, {'id': 'D004', 'name': 'Puerta Almacén', 'width_mm': 800.0, 'door_type': 'single', 'from_room': 'R002', 'to_room': 'R004', 'is_emergency_exit': False, 'fire_rating': 'RF_30'}]

{'doo

## 6. Conversational Interface

Finally, let's create a simple conversational interface where you can ask the agent anything!


In [8]:
# Create a conversational interface function
def chat_with_agent(question: str) -> str:
    """
    Chat with the ReAct agent.
    
    Args:
        question: The question to ask the agent
    
    Returns:
        String with the agent's response
    """
    print(f"\n💬 Question: {question}")
    print("=" * 80)
    
    # Create initial state
    initial_state = {
        "messages": [HumanMessage(content=question)],
        "iterations": 0
    }
    
    # Run the agent
    result = agent.invoke(initial_state)
    
    # Extract the final response
    final_response = ""
    for message in result["messages"]:
        if hasattr(message, 'content') and message.content and not message.content.startswith("You are"):
            final_response = message.content
    
    print(f"\n🤖 Agent Response:")
    print(final_response)
    print(f"\n📊 Completed in {result['iterations']} iterations")
    
    return final_response

print("✅ Conversational interface created")
print("💬 Use chat_with_agent('your question') to interact")


✅ Conversational interface created
💬 Use chat_with_agent('your question') to interact


In [9]:
# Demo the conversational interface
print("🎭 DEMO: Conversational Interface")
print("=" * 50)

# Example conversations
demo_questions = [
    "What building codes apply to emergency exit doors?",
    "Tell me about the largest room in the project",
    "Check if all doors in the project meet compliance requirements",
    "Analyze the building's circulation and connectivity"
]

for i, question in enumerate(demo_questions, 1):
    print(f"\n{'='*20} DEMO {i} {'='*20}")
    response = chat_with_agent(question)
    
    if i < len(demo_questions):
        input("\nPress Enter to continue to next demo...")

print("\n🎉 Demo completed!")
print("\n💡 Try your own questions:")
print("   chat_with_agent('Your question here')")


🎭 DEMO: Conversational Interface

==================== DEMO 1 ====================

💬 Question: What building codes apply to emergency exit doors?

🤖 Agent Response:
I am unable to query building codes without the RAG system being initialized. If you have specific building code references or regulations in mind, please provide them, and I can assist you in understanding how they apply to emergency exit doors.

📊 Completed in 5 iterations



Press Enter to continue to next demo... 



==================== DEMO 2 ====================

💬 Question: Tell me about the largest room in the project

🤖 Agent Response:
The largest room in the project is the reception room (Room ID: R001) located on the ground floor. It has an area of 20 square meters and is designated for reception use. The room has a fire rating of RF_60 and can accommodate up to 10 occupants. The calculated occupancy capacity is 2 individuals. The reception room is equipped with a door leading to the exterior, which serves as an emergency exit.

If you need more specific details or have any other questions regarding building compliance, feel free to ask!

📊 Completed in 7 iterations



Press Enter to continue to next demo... 



==================== DEMO 3 ====================

💬 Question: Check if all doors in the project meet compliance requirements

🤖 Agent Response:
All doors in the project meet the compliance requirements for minimum width:

- Door 'Puerta Principal' (D001) meets the minimum width requirement of 900mm according to CTE DB-SI Section 3.1.
- Door 'Puerta Recepción-Oficina' (D002) meets the minimum width requirement of 800mm according to CTE DB-SUA Section 2.1.
- Door 'Puerta Sala Reuniones' (D003) meets the minimum width requirement of 800mm according to CTE DB-SUA Section 2.1.
- Door 'Puerta Almacén' (D004) meets the minimum width requirement of 800mm according to CTE DB-SUA Section 2.1.

All doors are compliant with the minimum width regulations.

📊 Completed in 10 iterations



Press Enter to continue to next demo... 



==================== DEMO 4 ====================

💬 Question: Analyze the building's circulation and connectivity

🤖 Agent Response:
The building has the following doors:

1. **Puerta Principal (Main Door)**
   - Width: 900mm
   - Type: Single door
   - From: Exterior
   - To: Room R001
   - Emergency Exit: Yes
   - Fire Rating: RF_60

2. **Puerta Recepción-Oficina (Reception-Office Door)**
   - Width: 800mm
   - Type: Single door
   - From: Room R001
   - To: Room R002
   - Emergency Exit: No
   - Fire Rating: RF_60

3. **Puerta Sala Reuniones (Meeting Room Door)**
   - Width: 800mm
   - Type: Single door
   - From: Room R001
   - To: Room R003
   - Emergency Exit: No
   - Fire Rating: RF_60

4. **Puerta Almacén (Storage Room Door)**
   - Width: 800mm
   - Type: Single door
   - From: Room R002
   - To: Room R004
   - Emergency Exit: No
   - Fire Rating: RF_30

The circulation and connectivity in the building are facilitated through these doors, providing access between different roo

## 🚨 **Evacuation Distance Analysis Demo**

This demo shows the agent performing a complex compliance analysis by calculating evacuation distances for rooms in the building. This is a critical safety requirement in building codes.

### **🎯 What the Agent Will Do:**

1. **🧠 Reason** about evacuation requirements
2. **🔧 Use tools** to get room information and calculate distances
3. **📊 Analyze** compliance with building codes
4. **📝 Report** findings with recommendations

### **🔍 The ReAct Process in Action:**

- **Reasoning**: Agent thinks about what information it needs
- **Acting**: Agent uses tools to gather data and perform calculations
- **Observing**: Agent analyzes the results and determines next steps
- **Final Answer**: Agent provides comprehensive compliance analysis


In [11]:
# 🚨 EVACUATION DISTANCE ANALYSIS DEMO
# =====================================

print("🚨 EVACUATION DISTANCE ANALYSIS DEMO")
print("=" * 50)
print("This demo shows the agent calculating evacuation distances")
print("and analyzing compliance with building safety requirements.")
print("=" * 50)

# Demo question for evacuation analysis
demo_question = "Calculate the evacuation distance for room R001 (Recepción) and analyze if it meets building code requirements for emergency egress"

print(f"\n💬 Question: {demo_question}")
print("\n" + "="*80)
print("🔍 AGENT REASONING PROCESS (LIVE)")
print("="*80)

# Use the streaming version to show reasoning
response = chat_with_agent_streaming(demo_question)

print("\n🎉 Evacuation Analysis Demo Completed!")
print("\n💡 Key Takeaways:")
print("   - Agent automatically used multiple tools")
print("   - Performed complex calculations")
print("   - Analyzed compliance requirements")
print("   - Provided actionable recommendations")


🚨 EVACUATION DISTANCE ANALYSIS DEMO
This demo shows the agent calculating evacuation distances
and analyzing compliance with building safety requirements.

💬 Question: Calculate the evacuation distance for room R001 (Recepción) and analyze if it meets building code requirements for emergency egress

🔍 AGENT REASONING PROCESS (LIVE)


NameError: name 'chat_with_agent_streaming' is not defined

## 🏗️ **Comprehensive Building Compliance Demo**

This demo shows the agent performing a complete building compliance analysis, checking multiple aspects of the building design.

### **🎯 What the Agent Will Analyze:**

1. **🚪 Door Compliance** - Width requirements and accessibility
2. **🚨 Emergency Egress** - Evacuation routes and distances  
3. **📐 Room Layout** - Space utilization and circulation
4. **📋 Code Compliance** - Building regulation adherence
5. **💡 Recommendations** - Actionable improvement suggestions


In [14]:
# 🏗️ COMPREHENSIVE BUILDING COMPLIANCE DEMO
# ==========================================

print("🏗️ COMPREHENSIVE BUILDING COMPLIANCE DEMO")
print("=" * 50)
print("This demo shows the agent performing a complete")
print("building compliance analysis with multiple checks.")
print("=" * 50)

# Comprehensive compliance question
compliance_question = "Perform a comprehensive compliance analysis of this building. Check all doors for width compliance, analyze evacuation routes, and provide recommendations for any issues found."

print(f"\n💬 Question: {compliance_question}")
print("\n" + "="*80)
print("🔍 AGENT REASONING PROCESS (LIVE)")
print("="*80)

# Use the streaming version to show reasoning
response = chat_with_agent_streaming(compliance_question)

print("\n🎉 Comprehensive Compliance Analysis Completed!")
print("\n💡 Key Benefits of the ReAct Agent:")
print("   - 🧠 Autonomous reasoning about complex problems")
print("   - 🔧 Seamless tool integration and execution")
print("   - 📊 Multi-step analysis and calculations")
print("   - 📝 Professional compliance reporting")
print("   - 💡 Actionable recommendations")


🏗️ COMPREHENSIVE BUILDING COMPLIANCE DEMO
This demo shows the agent performing a complete
building compliance analysis with multiple checks.

💬 Question: Perform a comprehensive compliance analysis of this building. Check all doors for width compliance, analyze evacuation routes, and provide recommendations for any issues found.

🔍 AGENT REASONING PROCESS (LIVE)

💬 Question: Perform a comprehensive compliance analysis of this building. Check all doors for width compliance, analyze evacuation routes, and provide recommendations for any issues found.

🔍 AGENT REASONING PROCESS (LIVE)

🔄 ITERATION 1
----------------------------------------
🧠 Agent reasoning...
💭 Agent thought: ...

🔧 TOOL CALLS (1):

  1. 🔨 Executing: list_all_doors
     📝 Arguments: {}
     ✅ Result: [{'id': 'D001', 'name': 'Puerta Principal', 'width_mm': 900.0, 'door_type': 'single', 'from_room': '...

🔄 ITERATION 2
----------------------------------------
🧠 Agent reasoning...
💭 Agent thought: ...

🔧 TOOL CALLS (4):

  

## 🔍 **Understanding the ReAct Agent Process**

The ReAct (Reasoning + Acting) framework is a powerful approach that allows AI agents to:

### **🧠 Reasoning Phase**
- **Analyze** the problem and break it down into steps
- **Plan** which tools and information are needed
- **Think** about the best approach to solve the problem

### **🔧 Acting Phase**  
- **Execute** tools to gather data and perform calculations
- **Interact** with external systems (databases, APIs, calculations)
- **Collect** results from each tool execution

### **👁️ Observing Phase**
- **Evaluate** the results from tool executions
- **Decide** if more information is needed
- **Determine** when enough data has been collected

### **📝 Final Answer Phase**
- **Synthesize** all gathered information
- **Provide** comprehensive analysis and recommendations
- **Format** results in a professional, actionable manner

### **🎯 Key Benefits for AEC Professionals:**

1. **Transparency** - See exactly how the agent reaches conclusions
2. **Trust** - Verify the reasoning process and tool usage
3. **Efficiency** - Automate complex multi-step analyses
4. **Accuracy** - Consistent application of building codes and standards
5. **Scalability** - Handle multiple projects and compliance requirements


## 🎯 Summary

Congratulations! You've seen how all the components from the previous tutorials work together in a powerful ReAct agent:

### 🏗️ **What We Built**

1. **ReAct Agent**: Autonomous reasoning and acting system
2. **6 Specialized Tools**: Combining data access, calculations, and regulations
3. **Natural Language Interface**: Conversational compliance verification
4. **Complete Integration**: All tutorials working together seamlessly

### 🛠️ **Available Tools**

**Building Data** (from Tutorial 1):
- `get_room_info(room_id)` - Get detailed room information
- `get_door_info(door_id)` - Get detailed door information  
- `list_all_doors()` - List all doors in project

**Calculations** (from Tutorial 2):
- `check_door_width_compliance(door_id)` - Check door compliance
- `calculate_egress_distance(room_id)` - Calculate evacuation distances

**Regulations** (from Tutorial 3):
- `query_normativa(question)` - Query building codes with RAG

### 🚀 **Key Features**

- **Autonomous Reasoning**: Agent decides which tools to use
- **Multi-step Analysis**: Can combine multiple tools for complex tasks
- **Natural Language**: Conversational interface
- **Comprehensive**: Covers data, calculations, and regulations
- **Extensible**: Easy to add new tools and capabilities

### 💡 **Next Steps**

You can now:
1. **Ask Complex Questions**: The agent handles multi-step compliance checks
2. **Add More Tools**: Create tools for specific building types or regulations
3. **Integrate with CAD**: Connect to real CAD software
4. **Deploy**: Create a web interface or API

**This is the future of building compliance verification!** 🎉

### 🎓 **What You Learned**

- How ReAct agents work (Reasoning + Acting)
- How to use existing tools with LLMs
- How to build agent workflows with LangGraph
- How to integrate RAG for document querying
- How to create conversational AI interfaces

**The agent can now autonomously verify building compliance using all the tools we've built!**


In [13]:
# Create a streaming version that shows the agent's reasoning
def chat_with_agent_streaming(question: str) -> str:
    """
    Chat with the ReAct agent showing real-time reasoning and tool usage.
    
    Args:
        question: The question to ask the agent
    
    Returns:
        String with the agent's response
    """
    print(f"\n💬 Question: {question}")
    print("\n" + "="*60)
    print("🔍 AGENT REASONING PROCESS (LIVE)")
    print("="*60)
    
    # Load project data if not already loaded
    from src.agent.tools import load_project_data
    project_file = Path("data/extracted/tutorial_example.json")
    if project_file.exists():
        load_project_data(project_file)
    
    # Create system prompt
    system_prompt = f"""You are an expert AEC (Architecture, Engineering, Construction) compliance verification agent.

Your task: {question}

You have access to the following tools:
- get_room_info: Get detailed information about a room
- get_door_info: Get detailed information about a door  
- list_all_doors: List all doors in the project
- check_door_width_compliance: Check if a door meets width requirements
- query_normativa: Query building codes and regulations
- calculate_egress_distance: Calculate evacuation distances

Use these tools to gather information and provide comprehensive answers about building compliance.

When you have enough information, provide a clear, detailed response without making additional tool calls."""

    # Start conversation
    conversation = [
        HumanMessage(content=system_prompt),
        HumanMessage(content=question)
    ]
    
    # Run ReAct loop with live updates
    for iteration in range(10):  # Max 10 iterations
        print(f"\n🔄 ITERATION {iteration + 1}")
        print("-" * 40)
        
        try:
            # Get agent response
            print("🧠 Agent reasoning...")
            response = agent.llm_with_tools.invoke(conversation)
            conversation.append(response)
            
            print(f"💭 Agent thought: {response.content[:200]}...")
            
            # Check if agent made tool calls
            if hasattr(response, 'tool_calls') and response.tool_calls:
                print(f"\n🔧 TOOL CALLS ({len(response.tool_calls)}):")
                
                # Execute tool calls
                for i, tool_call in enumerate(response.tool_calls, 1):
                    tool_name = tool_call['name']
                    tool_args = tool_call['args']
                    tool_id = tool_call['id']
                    
                    print(f"\n  {i}. 🔨 Executing: {tool_name}")
                    print(f"     📝 Arguments: {tool_args}")
                    
                    # Execute the tool
                    if tool_name in agent.tools:
                        try:
                            result = agent.tools[tool_name].invoke(tool_args)
                            print(f"     ✅ Result: {str(result)[:100]}...")
                            
                            # Create tool message
                            from langchain_core.messages import ToolMessage
                            tool_message = ToolMessage(
                                content=str(result),
                                tool_call_id=tool_id
                            )
                            conversation.append(tool_message)
                            
                        except Exception as e:
                            print(f"     ❌ Error: {str(e)}")
                            tool_message = ToolMessage(
                                content=f"Error: {str(e)}",
                                tool_call_id=tool_id
                            )
                            conversation.append(tool_message)
                    else:
                        print(f"     ❌ Unknown tool: {tool_name}")
            else:
                # No tool calls, agent provided final answer
                print(f"\n✅ FINAL ANSWER:")
                print(f"📝 {response.content}")
                break
                
        except Exception as e:
            print(f"\n❌ Error in iteration {iteration + 1}: {str(e)}")
            break
    
    print(f"\n📊 Completed in {len(conversation)} steps")
    return conversation[-1].content if conversation else "No response generated"

print("✅ Streaming interface created")
print("🔍 Use chat_with_agent_streaming('your question') for live reasoning")


✅ Streaming interface created
🔍 Use chat_with_agent_streaming('your question') for live reasoning


## 🚀 Quick Start Guide

If you want to jump straight to using the agent, here's a quick start:

### 1. **Run the setup cells** (cells 3-4) to:
   - Import all required libraries
   - Load project data
   - Set up RAG (if available)
   - Create the agent

### 2. **Test individual tools** (cell 6) to verify everything works

### 3. **Try the agent** with simple queries:
```python
# Simple query
result = agent.invoke({
    "messages": [HumanMessage(content="List all doors in the project")]
})

# Complex query
result = agent.invoke({
    "messages": [HumanMessage(content="Check compliance for all doors and calculate evacuation distances")]
})
```

### 4. **Use the conversational interface**:
```python
# Interactive chat
response = chat_with_agent("What are the building code requirements for emergency exits?")
```

### 🔧 **Troubleshooting**

#### **Missing Dependencies**
If you get import errors, install the required packages:

```bash
# Install all dependencies at once
pip install -r requirements.txt

# Or install individually
pip install langgraph
pip install langchain-openai
pip install langchain-community
pip install langchain-huggingface
pip install langchain-core
```

#### **Version Compatibility Issues**
If you get "cannot import name 'content'" error:

```bash
# Option 1: Downgrade langchain-openai
pip install langchain-openai==0.2.8

# Option 2: Upgrade both packages
pip install --upgrade langchain-openai langchain-core
```

#### **Other Issues**
- **Import errors**: Make sure you're running cells in order
- **Agent not defined**: Run the setup cells first
- **No project data**: Check that `data/extracted/tutorial_example.json` exists
- **RAG not working**: Run `python scripts/create_vectorstore.py` first


In [ ]:
# Demo the streaming conversational interface
print("🎭 DEMO: Live Agent Reasoning")
print("=" * 60)
print("🔍 This demo shows the agent's step-by-step reasoning process")
print("=" * 60)

# Example with streaming
demo_question = "List all doors in the project and check their compliance"

print(f"\n💬 Question: {demo_question}")
print("\n" + "="*60)

# Use streaming version to show reasoning
response = chat_with_agent_streaming(demo_question)

print("\n🎉 Demo completed!")
print("\n💡 Try your own questions:")
print("   chat_with_agent_streaming('Your question here') - for live reasoning")
print("   chat_with_agent('Your question here') - for simple response")


## 🔍 **Live Agent Reasoning**

The streaming interface shows you exactly how the ReAct agent works:

### **🧠 Reasoning Process**
1. **Agent thinks** - The LLM analyzes the question and decides what to do
2. **Tool selection** - The agent chooses which tools to use
3. **Tool execution** - Tools are called with specific arguments
4. **Result processing** - The agent analyzes tool results
5. **Final answer** - The agent synthesizes everything into a response

### **🔧 Available Tools**
- **Data Tools**: `get_room_info`, `get_door_info`, `list_all_doors`
- **Calculation Tools**: `check_door_width_compliance`, `calculate_egress_distance`
- **RAG Tool**: `query_normativa` (for building codes)

### **💡 Usage Examples**

```python
# See the agent's reasoning process
chat_with_agent_streaming("What are the building requirements for emergency exits?")

# Simple response (no reasoning shown)
chat_with_agent("List all rooms in the project")
```

### **🎯 Key Benefits**
- **Transparency** - See exactly how the agent thinks
- **Debugging** - Understand why the agent made certain decisions
- **Learning** - Learn the ReAct framework by watching it work
- **Trust** - Verify the agent's reasoning process


## 🚀 Installation Guide

If you're missing dependencies, here's how to install them:

### **Quick Install (Recommended)**
```bash
pip install -r requirements.txt
```

### **Individual Package Install**
If you prefer to install packages individually:

```bash
# Core LangChain packages
pip install langchain-core==0.2.38
pip install langchain==0.2.16
pip install langchain-community==0.2.16
pip install langchain-openai==0.1.7
pip install langgraph==0.2.16

# RAG and embeddings
pip install langchain-huggingface==0.1.0
pip install chromadb==0.4.22
pip install sentence-transformers==2.3.1

# Other dependencies
pip install pydantic==2.5.3
pip install shapely==2.0.2
pip install networkx==3.2.1
pip install numpy==1.26.4
```

### **Version Compatibility Fix**
If you encounter the "cannot import name 'content'" error:

```bash
# Option 1: Use the exact versions from requirements.txt
pip install langchain-openai==0.1.7 langchain-core==0.2.38

# Option 2: If that doesn't work, try downgrading
pip install langchain-openai==0.2.8
```

### **Verify Installation**
After installation, restart your Jupyter kernel and run the setup cells again.

## 🔑 API Key Setup

To use the ReAct agent, you need to set up your OpenAI API key:

### **Option 1: Environment Variable (Recommended)**
```bash
export OPENAI_API_KEY="your-api-key-here"
```

### **Option 2: Create .env file**
Create a `.env` file in the project root:
```
OPENAI_API_KEY=your-api-key-here
```

### **Option 3: Set in Jupyter**
```python
import os
os.environ["OPENAI_API_KEY"] = "your-api-key-here"
```

## 📚 RAG System Setup

To enable building code querying, create the RAG vectorstore:

```bash
python scripts/create_vectorstore.py
```

This will:
- Load PDF documents from `data/normativa/`
- Create embeddings using HuggingFace models
- Build a ChromaDB vectorstore for document retrieval


In [ ]:
# Quick setup for testing without full agent
print("🔧 Quick Setup for Testing")
print("=" * 50)

# Set up API key if not already set
import os
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️ OpenAI API key not found. You can:")
    print("1. Set it as environment variable: export OPENAI_API_KEY='your-key'")
    print("2. Create a .env file with: OPENAI_API_KEY=your-key")
    print("3. Set it in Python: os.environ['OPENAI_API_KEY'] = 'your-key'")
    print("\n💡 For now, you can still test the individual tools below!")
    
    # Option to set API key directly in the notebook (for testing)
    print("\n🔑 Quick API Key Setup (for testing only):")
    print("   Uncomment and modify the line below with your actual API key:")
    print("   # os.environ['OPENAI_API_KEY'] = 'your-actual-api-key-here'")
else:
    print("✅ OpenAI API key found")

# Check if we have project data
if 'project_file' in locals() and project_file:
    print(f"✅ Project data available: {project_file}")
    print("🎉 Great! You can now test all the individual tools!")
else:
    print("⚠️ No project data loaded. The tools will show error messages.")
    print("   This is expected if the file path couldn't be resolved.")

print("\n🎯 You can now:")
print("   - Test individual tools (next section) ✅")
print("   - Learn about the ReAct framework ✅")
print("   - See how the agent would work conceptually ✅")
print("   - Set up API key to test the full agent")


In [ ]:
# 🎯 Status Check
print("🔍 Final Status Check")
print("=" * 50)

# Check what's available
if 'agent' in locals():
    print("✅ ReAct Agent: Ready")
    print("🔧 Tool calling issue has been fixed!")
else:
    print("⚠️ ReAct Agent: Not available")

if 'project_file' in locals() and project_file:
    print("✅ Project Data: Loaded")
else:
    print("⚠️ Project Data: Not loaded")

if 'rag_available' in locals() and rag_available:
    print("✅ RAG System: Connected")
else:
    print("⚠️ RAG System: Not available")

print("\n🎉 You're ready to explore the tutorial!")
print("   - Individual tools will work with project data")
print("   - Agent examples will work if API key is set")
print("   - RAG queries will work if vectorstore is available")
print("   - Tool calling errors have been resolved!")


# 🤖 Tutorial 4: ReAct Agent - Autonomous Compliance Verification

## 🎯 What You'll Learn

In this tutorial, you'll see how all the components from the previous tutorials work together in a **ReAct Agent** that can:

1. **🧠 Reason** about building compliance problems autonomously
2. **🔧 Use tools** to access data, perform calculations, and query regulations
3. **📚 Query documents** using the RAG system from Tutorial 3
4. **🏗️ Analyze buildings** using calculations from Tutorial 2
5. **💬 Have conversations** about compliance in natural language

## 📋 Table of Contents

1. [What is a ReAct Agent?](#1-what-is-a-react-agent)
2. [Setting Up the Agent](#2-setting-up-the-agent)
3. [Understanding the Tools](#3-understanding-the-tools)
4. [Testing Simple Queries](#4-testing-simple-queries)
5. [Complex Compliance Verification](#5-complex-compliance-verification)
6. [Conversational Interface](#6-conversational-interface)

---


## 1. What is a ReAct Agent?

**ReAct** = **Rea**soning + **Act**ing

A ReAct agent is an AI system that can:
1. 🧠 **Think** about what it needs to do
2. 🔧 **Choose** which tools to use
3. ⚡ **Execute** the tools
4. 📊 **Observe** the results
5. 🔁 **Repeat** until it has the answer

### How it works:

```
User: "Check if all doors meet requirements"
     ↓
Agent: "I need to list all doors first"
     ↓
Tool: list_all_doors()
     ↓
Agent: "Now I'll check each door's compliance"
     ↓
Tool: check_door_width_compliance("D001")
     ↓
Agent: "Let me query the regulations for requirements"
     ↓
Tool: query_normativa("minimum door width")
     ↓
Agent: "Here's my final answer..."
```

The agent **autonomously** decides what to do and uses the right tools to get the information it needs!


## 2. Setting Up the Agent

Let's set up our ReAct agent using the components we've already built!


In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
from langchain_core.messages import HumanMessage

# Add the project root to Python path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

# Import our existing agent components
from src.agent.graph import create_compliance_agent
from src.agent.tools import load_project_data, set_vectorstore_manager

print("✅ Libraries imported successfully!")
print("🤖 Ready to use our ReAct agent")


In [ ]:
# Load project data (use example dataset from Tutorial 1/2)
project_file = Path("data/extracted/tutorial_example.json")
try:
    load_project_data(project_file)
    print(f"✅ Project data loaded: {project_file}")
except Exception as e:
    print(f"❌ Error loading project data: {e}")
    print("Make sure the file exists and is valid JSON")

# Setup RAG for query_normativa tool if available
rag_available = False
vectorstore_path = Path("vectorstore/normativa_db")
try:
    from src.rag.vectorstore_manager import VectorstoreManager
    from src.rag.qa_chain import create_qa_chain
    rag_available = vectorstore_path.exists() and any(vectorstore_path.iterdir())
except Exception:
    rag_available = False

if rag_available:
    try:
        rag = VectorstoreManager(vectorstore_path)
        rag.load_existing()
        retriever = rag.get_retriever(k=3)
        qa_chain = create_qa_chain(retriever)

        class RAGWrapper:
            def query(self, question):
                return qa_chain({"query": question})

        set_vectorstore_manager(RAGWrapper())
        print("✅ RAG system connected")
    except Exception as e:
        print("⚠️ Could not initialize RAG:", e)
        print("   Tip: Create the vectorstore before the demo.")
        print("   For setup, see docs or run: python scripts/create_vectorstore.py")
else:
    print("ℹ️ RAG vectorstore not found. Skipping CTE querying in this run.")
    print("   To enable, run: python scripts/create_vectorstore.py")

# Create agent
try:
    agent = create_compliance_agent()
    print("✅ Agent created")
    print("\n🤖 Ready to verify compliance!")
except Exception as e:
    print(f"❌ Error creating agent: {e}")
    print("Make sure all dependencies are installed and the agent module is working")


## 3. Understanding the Tools

Our ReAct agent has access to 6 powerful tools that combine all the functionality from the previous tutorials:

### 🏗️ **Data Tools** (from Tutorial 1)
- **`get_room_info(room_id)`** - Get detailed room information
- **`get_door_info(door_id)`** - Get detailed door information  
- **`list_all_doors()`** - List all doors in the project

### 📐 **Calculation Tools** (from Tutorial 2)
- **`check_door_width_compliance(door_id)`** - Check door width compliance
- **`calculate_egress_distance(room_id)`** - Calculate evacuation distances

### 📚 **RAG Tool** (from Tutorial 3)
- **`query_normativa(question)`** - Query Spanish building codes (CTE)

Let's test these tools individually first to make sure they work!


In [ ]:
# Test the tools individually first
from src.agent.tools import (
    get_room_info, get_door_info, list_all_doors,
    check_door_width_compliance, query_normativa, calculate_egress_distance
)

print("🧪 Testing Tools Individually")
print("=" * 50)

# Test 1: List all doors
print("\n1. Testing list_all_doors.invoke({}):")
try:
    doors = list_all_doors.invoke({})
    print(f"   Found {len(doors)} doors:")
    for door in doors[:3]:  # Show first 3
        print(f"   - {door.get('id', 'Unknown')}: {door.get('name', 'N/A')}")
    if len(doors) > 3:
        print(f"   ... and {len(doors) - 3} more")
    print("   ✅ list_all_doors.invoke({}) works!")
except Exception as e:
    print(f"   ❌ Error: {e}")

# Test 2: Get room info
print("\n2. Testing get_room_info():")
try:
    room_info = get_room_info.invoke({"room_id": "R001"})
    if "error" not in room_info:
        print(f"   Room: {room_info.get('name', 'Unknown')}")
        print(f"   Area: {room_info.get('area_sqm', 'Unknown')} m²")
        print(f"   Use: {room_info.get('use', 'Unknown')}")
        print("   ✅ get_room_info() works!")
    else:
        print(f"   ❌ Error: {room_info['error']}")
except Exception as e:
    print(f"   ❌ Error: {e}")

# Test 3: Check door compliance
print("\n3. Testing check_door_width_compliance():")
try:
    compliance = check_door_width_compliance.invoke({"door_id": "D001"})
    if "error" not in compliance:
        print(f"   Door: {compliance.get('door_id', 'Unknown')}")
        print(f"   Width: {compliance.get('clear_width_mm', 'Unknown')} mm")
        print(f"   Status: {compliance.get('compliance_status', 'Unknown')}")
        print("   ✅ check_door_width_compliance() works!")
    else:
        print(f"   ❌ Error: {compliance['error']}")
except Exception as e:
    print(f"   ❌ Error: {e}")

print("\n✅ Tool testing complete!")


## 4. Testing Simple Queries

Now let's test the agent with simple queries to see it in action!


In [ ]:
# Example 1: List all doors
print("="*70)
print("EXAMPLE 1: List all doors")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="List all doors in the project")],
    "iterations": 0
})

# Print agent's response
for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)

print(f"\n✅ Completed in {result['iterations']} iterations")


In [ ]:
# Example 2: Check door compliance
print("="*70)
print("EXAMPLE 2: Check door compliance")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="Check if all doors meet minimum width requirements")],
    "iterations": 0
})

# Print agent's reasoning and answer
for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)
        print()

print(f"✅ Completed in {result['iterations']} iterations")


## 5. Complex Compliance Verification

Now let's see the agent handle more complex tasks that require multiple tools and reasoning!


In [ ]:
# Example 3: Complex compliance check
print("="*70)
print("EXAMPLE 3: Complex Compliance Check")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="""
    Perform a comprehensive compliance check:
    1. List all doors
    2. Check each door's width compliance
    3. Calculate evacuation distances from all rooms
    4. Summarize findings with specific non-compliant items
    """)],
    "iterations": 0
})

for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)
        print()

print(f"✅ Completed in {result['iterations']} iterations")


## 6. Conversational Interface

Finally, let's create a simple conversational interface where you can ask the agent anything!


In [ ]:
# Create a conversational interface function
def chat_with_agent(question: str) -> str:
    """
    Chat with the ReAct agent.
    
    Args:
        question: The question to ask the agent
    
    Returns:
        String with the agent's response
    """
    print(f"\n💬 Question: {question}")
    print("=" * 80)
    
    # Create initial state
    initial_state = {
        "messages": [HumanMessage(content=question)],
        "iterations": 0
    }
    
    # Run the agent
    result = agent.invoke(initial_state)
    
    # Extract the final response
    final_response = ""
    for message in result["messages"]:
        if hasattr(message, 'content') and message.content and not message.content.startswith("You are"):
            final_response = message.content
    
    print(f"\n🤖 Agent Response:")
    print(final_response)
    print(f"\n📊 Completed in {result['iterations']} iterations")
    
    return final_response

print("✅ Conversational interface created")
print("💬 Use chat_with_agent('your question') to interact")


In [ ]:
# Demo the conversational interface
print("🎭 DEMO: Conversational Interface")
print("=" * 50)

# Example conversations
demo_questions = [
    "What building codes apply to emergency exit doors?",
    "Tell me about the largest room in the project",
    "Check if all doors in the project meet compliance requirements",
    "Analyze the building's circulation and connectivity"
]

for i, question in enumerate(demo_questions, 1):
    print(f"\n{'='*20} DEMO {i} {'='*20}")
    response = chat_with_agent(question)
    
    if i < len(demo_questions):
        input("\nPress Enter to continue to next demo...")

print("\n🎉 Demo completed!")
print("\n💡 Try your own questions:")
print("   chat_with_agent('Your question here')")


## 🎯 Summary

Congratulations! You've seen how all the components from the previous tutorials work together in a powerful ReAct agent:

### 🏗️ **What We Built**

1. **ReAct Agent**: Autonomous reasoning and acting system
2. **6 Specialized Tools**: Combining data access, calculations, and regulations
3. **Natural Language Interface**: Conversational compliance verification
4. **Complete Integration**: All tutorials working together seamlessly

### 🛠️ **Available Tools**

**Building Data** (from Tutorial 1):
- `get_room_info(room_id)` - Get detailed room information
- `get_door_info(door_id)` - Get detailed door information  
- `list_all_doors()` - List all doors in project

**Calculations** (from Tutorial 2):
- `check_door_width_compliance(door_id)` - Check door compliance
- `calculate_egress_distance(room_id)` - Calculate evacuation distances

**Regulations** (from Tutorial 3):
- `query_normativa(question)` - Query building codes with RAG

### 🚀 **Key Features**

- **Autonomous Reasoning**: Agent decides which tools to use
- **Multi-step Analysis**: Can combine multiple tools for complex tasks
- **Natural Language**: Conversational interface
- **Comprehensive**: Covers data, calculations, and regulations
- **Extensible**: Easy to add new tools and capabilities

### 💡 **Next Steps**

You can now:
1. **Ask Complex Questions**: The agent handles multi-step compliance checks
2. **Add More Tools**: Create tools for specific building types or regulations
3. **Integrate with CAD**: Connect to real CAD software
4. **Deploy**: Create a web interface or API

**This is the future of building compliance verification!** 🎉

### 🎓 **What You Learned**

- How ReAct agents work (Reasoning + Acting)
- How to use existing tools with LLMs
- How to build agent workflows with LangGraph
- How to integrate RAG for document querying
- How to create conversational AI interfaces

**The agent can now autonomously verify building compliance using all the tools we've built!**


# 🤖 Tutorial 4: ReAct Agent - Autonomous Compliance Verification

## 🎯 What You'll Learn

In this tutorial, you'll see how all the components from the previous tutorials work together in a **ReAct Agent** that can:

1. **🧠 Reason** about building compliance problems autonomously
2. **🔧 Use tools** to access data, perform calculations, and query regulations
3. **📚 Query documents** using the RAG system from Tutorial 3
4. **🏗️ Analyze buildings** using calculations from Tutorial 2
5. **💬 Have conversations** about compliance in natural language

## 📋 Table of Contents

1. [What is a ReAct Agent?](#1-what-is-a-react-agent)
2. [Setting Up the Agent](#2-setting-up-the-agent)
3. [Understanding the Tools](#3-understanding-the-tools)
4. [Testing Simple Queries](#4-testing-simple-queries)
5. [Complex Compliance Verification](#5-complex-compliance-verification)
6. [Conversational Interface](#6-conversational-interface)

---


## 1. What is a ReAct Agent?

**ReAct** = **Rea**soning + **Act**ing

A ReAct agent is an AI system that can:
1. 🧠 **Think** about what it needs to do
2. 🔧 **Choose** which tools to use
3. ⚡ **Execute** the tools
4. 📊 **Observe** the results
5. 🔁 **Repeat** until it has the answer

### How it works:

```
User: "Check if all doors meet requirements"
     ↓
Agent: "I need to list all doors first"
     ↓
Tool: list_all_doors()
     ↓
Agent: "Now I'll check each door's compliance"
     ↓
Tool: check_door_width_compliance("D001")
     ↓
Agent: "Let me query the regulations for requirements"
     ↓
Tool: query_normativa("minimum door width")
     ↓
Agent: "Here's my final answer..."
```

The agent **autonomously** decides what to do and uses the right tools to get the information it needs!


## 2. Setting Up the Agent

Let's set up our ReAct agent using the components we've already built!


In [ ]:
# Import required libraries
import sys
from pathlib import Path
from langchain_core.messages import HumanMessage

# Add src to path for imports
sys.path.append('.')

# Import our existing agent components
from src.agent.graph import create_compliance_agent
from src.agent.tools import load_project_data, set_vectorstore_manager

print("✅ Libraries imported successfully!")
print("🤖 Ready to use our ReAct agent")


In [ ]:
# Load project data (use example dataset from Tutorial 1/2)
project_file = Path("data/extracted/tutorial_example.json")
load_project_data(project_file)
print(f"✅ Project data loaded: {project_file}")

# Setup RAG for query_normativa tool if available
rag_available = False
vectorstore_path = Path("vectorstore/normativa_db")
try:
    from src.rag.vectorstore_manager import VectorstoreManager
    from src.rag.qa_chain import create_qa_chain
    rag_available = vectorstore_path.exists() and any(vectorstore_path.iterdir())
except Exception:
    rag_available = False

if rag_available:
    try:
        rag = VectorstoreManager(vectorstore_path)
        rag.load_existing()
        retriever = rag.get_retriever(k=3)
        qa_chain = create_qa_chain(retriever)

        class RAGWrapper:
            def query(self, question):
                return qa_chain({"query": question})

        set_vectorstore_manager(RAGWrapper())
        print("✅ RAG system connected")
    except Exception as e:
        print("⚠️ Could not initialize RAG:", e)
        print("   Tip: Create the vectorstore before the demo.")
        print("   For setup, see docs or run: python scripts/create_vectorstore.py")
else:
    print("ℹ️ RAG vectorstore not found. Skipping CTE querying in this run.")
    print("   To enable, run: python scripts/create_vectorstore.py")

# Create agent
agent = create_compliance_agent()
print("✅ Agent created")
print("\n🤖 Ready to verify compliance!")


## 3. Understanding the Tools

Our ReAct agent has access to 6 powerful tools that combine all the functionality from the previous tutorials:

### 🏗️ **Data Tools** (from Tutorial 1)
- **`get_room_info(room_id)`** - Get detailed room information
- **`get_door_info(door_id)`** - Get detailed door information  
- **`list_all_doors()`** - List all doors in the project

### 📐 **Calculation Tools** (from Tutorial 2)
- **`check_door_width_compliance(door_id)`** - Check door width compliance
- **`calculate_egress_distance(room_id)`** - Calculate evacuation distances

### 📚 **RAG Tool** (from Tutorial 3)
- **`query_normativa(question)`** - Query Spanish building codes (CTE)

The agent can use these tools **autonomously** to answer complex compliance questions!


## 4. Testing Simple Queries

Let's start with some simple queries to see the agent in action!


In [ ]:
# Example 1: List all doors
print("="*70)
print("EXAMPLE 1: List all doors")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="List all doors in the project")],
    "iterations": 0
})

# Print agent's response
for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)

print(f"\n✅ Completed in {result['iterations']} iterations")


In [ ]:
# Example 2: Check door compliance
print("="*70)
print("EXAMPLE 2: Check door compliance")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="Check if all doors meet minimum width requirements")],
    "iterations": 0
})

# Print agent's reasoning and answer
for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)
        print()

print(f"✅ Completed in {result['iterations']} iterations")


## 5. Complex Compliance Verification

Now let's see the agent handle more complex tasks that require multiple tools and reasoning!


In [ ]:
# Example 3: Complex compliance check
print("="*70)
print("EXAMPLE 3: Complex Compliance Check")
print("="*70 + "\n")

result = agent.invoke({
    "messages": [HumanMessage(content="""
    Perform a comprehensive compliance check:
    1. List all doors
    2. Check each door's width compliance
    3. Calculate evacuation distances from all rooms
    4. Summarize findings with specific non-compliant items
    """)],
    "iterations": 0
})

for msg in result["messages"]:
    if hasattr(msg, 'content') and msg.content and not msg.content.startswith("You are"):
        print(msg.content)
        print()

print(f"✅ Completed in {result['iterations']} iterations")


## 6. Conversational Interface

Finally, let's create a simple conversational interface where you can ask the agent anything!


In [ ]:
# Create a conversational interface function
def chat_with_agent(question: str) -> str:
    """
    Chat with the ReAct agent.
    
    Args:
        question: The question to ask the agent
    
    Returns:
        String with the agent's response
    """
    print(f"\n💬 Question: {question}")
    print("=" * 80)
    
    # Create initial state
    initial_state = {
        "messages": [HumanMessage(content=question)],
        "iterations": 0
    }
    
    # Run the agent
    result = agent.invoke(initial_state)
    
    # Extract the final response
    final_response = ""
    for message in result["messages"]:
        if hasattr(message, 'content') and message.content and not message.content.startswith("You are"):
            final_response = message.content
    
    print(f"\n🤖 Agent Response:")
    print(final_response)
    print(f"\n📊 Completed in {result['iterations']} iterations")
    
    return final_response

print("✅ Conversational interface created")
print("💬 Use chat_with_agent('your question') to interact")


In [ ]:
# Demo the conversational interface
print("🎭 DEMO: Conversational Interface")
print("=" * 50)

# Example conversations
demo_questions = [
    "What building codes apply to emergency exit doors?",
    "Tell me about the largest room in the project",
    "Check if all doors in the project meet compliance requirements",
    "Analyze the building's circulation and connectivity"
]

for i, question in enumerate(demo_questions, 1):
    print(f"\n{'='*20} DEMO {i} {'='*20}")
    response = chat_with_agent(question)
    
    if i < len(demo_questions):
        input("\nPress Enter to continue to next demo...")

print("\n🎉 Demo completed!")
print("\n💡 Try your own questions:")
print("   chat_with_agent('Your question here')")


## 🎯 Summary

Congratulations! You've seen how all the components from the previous tutorials work together in a powerful ReAct agent:

### 🏗️ **What We Built**

1. **ReAct Agent**: Autonomous reasoning and acting system
2. **6 Specialized Tools**: Combining data access, calculations, and regulations
3. **Natural Language Interface**: Conversational compliance verification
4. **Complete Integration**: All tutorials working together seamlessly

### 🛠️ **Available Tools**

**Building Data** (from Tutorial 1):
- `get_room_info(room_id)` - Get detailed room information
- `get_door_info(door_id)` - Get detailed door information  
- `list_all_doors()` - List all doors in project

**Calculations** (from Tutorial 2):
- `check_door_width_compliance(door_id)` - Check door compliance
- `calculate_egress_distance(room_id)` - Calculate evacuation distances

**Regulations** (from Tutorial 3):
- `query_normativa(question)` - Query building codes with RAG

### 🚀 **Key Features**

- **Autonomous Reasoning**: Agent decides which tools to use
- **Multi-step Analysis**: Can combine multiple tools for complex tasks
- **Natural Language**: Conversational interface
- **Comprehensive**: Covers data, calculations, and regulations
- **Extensible**: Easy to add new tools and capabilities

### 💡 **Next Steps**

You can now:
1. **Ask Complex Questions**: The agent handles multi-step compliance checks
2. **Add More Tools**: Create tools for specific building types or regulations
3. **Integrate with CAD**: Connect to real CAD software
4. **Deploy**: Create a web interface or API

**This is the future of building compliance verification!** 🎉

### 🎓 **What You Learned**

- How ReAct agents work (Reasoning + Acting)
- How to use existing tools with LLMs
- How to build agent workflows with LangGraph
- How to integrate RAG for document querying
- How to create conversational AI interfaces

**The agent can now autonomously verify building compliance using all the tools we've built!**
